##Service Call Forecasting Data visualizations

###Count of failure_date grouped by period year

In [0]:
%sql
select extract(year from period_as_date) year_period, count(failure_date) from team_group_quality.service_call_forecasting group by extract(year from period_as_date) order by year_period

Databricks visualization. Run in Databricks to view.

###Count of failure date grouped by product line

In [0]:
%sql
select product_line, count(failure_date) from team_group_quality.service_call_forecasting group by product_line order by product_line

Databricks visualization. Run in Databricks to view.

###Count of failure date in each market over time

In [0]:
%sql
select extract(year from period_as_date) year_period, market, count(failure_date) from team_group_quality.service_call_forecasting group by extract(year from period_as_date), market order by year_period

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select pnc_sku, extract(year from period_as_date), count(failure_date) from team_group_quality.service_call_forecasting where pnc_sku in (select pnc_sku from team_group_quality.service_call_forecasting limit 50) group by pnc_sku,
extract(year from period_as_date) 

Databricks visualization. Run in Databricks to view.

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib

base = spark.sql("select extract(year from period_as_date) year_period, pnc_sku , count(sell_out_date) from team_group_quality.service_call_forecasting group by extract(year from period_as_date), pnc_sku")


#test_df = pd.read_csv(base)

test_df = base.toPandas()

plt.figure(figsize=(8,8))

sns.barplot(x = 'year_period', y = 'count(sell_out_date)', data = test_df)

plt.show()




###Count of distinct pnc_sku over sell_out_year for the major product lines('Dish Care','Fabric Care', 'Food Preparation', 'Food Preservation'))

In [0]:

base2 = spark.sql("select extract(year from sell_out_date) year_sell_out_date, product_line, count(distinct pnc_sku) count_distinct_pnc_sku from team_group_quality.service_call_forecasting where product_line in ('Dish Care','Fabric Care', 'Food Preparation', 'Food Preservation') group by extract(year from sell_out_date), product_line having extract(year from sell_out_date) between '2002' and '2023'")

test_df2 = base2.toPandas()


plt.figure(figsize=(20,20))

sns.relplot(data=test_df2, x='year_sell_out_date', y='count_distinct_pnc_sku', kind='line', hue='product_line').set(title="number of pnc_sku over years(sell_out_date) grouped by product line")

#sns.lineplot(x = 'year_sell_out_date', y = 'count(pnc_sku)', data = test_df2 )

plt.show()


###Count of distinct pnc_sku over failure_year for the major product lines('Dish Care','Fabric Care', 'Food Preparation', 'Food Preservation'))

In [0]:

base3 = spark.sql("select extract(year from failure_date) year_failure_date, product_line, count(distinct pnc_sku) count_distinct_pnc_sku from team_group_quality.service_call_forecasting where product_line in ('Dish Care','Fabric Care', 'Food Preparation', 'Food Preservation') group by extract(year from failure_date), product_line ")

test_df3 = base3.toPandas()


plt.figure(figsize=(20,20))

sns.relplot(data=test_df3, x='year_failure_date', y='count_distinct_pnc_sku', kind='line', hue='product_line').set(title="number of pnc_sku over years(failure_date) grouped by product line")

#sns.lineplot(x = 'year_sell_out_date', y = 'count(pnc_sku)', data = test_df2 )

plt.show()


In [0]:
%sql
select extract(year from sell_out_date) year_sell_out_date, count(pnc_sku) from team_group_quality.service_call_forecasting group by extract(year from sell_out_date) having extract(year from sell_out_date) between '2002' and '2023' order by year_sell_out_date

Databricks visualization. Run in Databricks to view.

###EDA

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#### Number of calls overtime (overall)

In [0]:
df = spark.sql("select * from team_group_quality.service_call_forecasting")
ncalls_overtime = df.groupby('period_as_date').agg({'qty_serv': 'sum'}).toPandas()
ncalls_overtime = ncalls_overtime.sort_values('period_as_date', ascending=True)
ncalls_overtime

In [0]:
sns.lineplot(data=ncalls_overtime, x="period_as_date", y="sum(qty_serv)").set(title="Overall # calls over time")

#### Number of calls over time per product line (showing only major appliances atm.)

In [0]:
df_dishcare = spark.sql("select period_as_date, qty_serv, product_line from team_group_quality.service_call_forecasting where product_line in ('Dish Care')")
ncalls_overtime_dishcare = df_dishcare.groupby('period_as_date').agg({'qty_serv': 'sum'}).toPandas()
ncalls_overtime_dishcare['product_line'] = "Dish Care"

df_fabriccare = spark.sql("select period_as_date, qty_serv, product_line from team_group_quality.service_call_forecasting where product_line in ('Fabric Care')")
ncalls_overtime_fabriccare = df_fabriccare.groupby('period_as_date').agg({'qty_serv': 'sum'}).toPandas()
ncalls_overtime_fabriccare['product_line'] = "Fabric Care"

df_foodprep = spark.sql("select period_as_date, qty_serv, product_line from team_group_quality.service_call_forecasting where product_line in ('Food Preparation')")
ncalls_overtime_foodprep = df_foodprep.groupby('period_as_date').agg({'qty_serv': 'sum'}).toPandas()
ncalls_overtime_foodprep['product_line'] = "Food Preparation"

df_foodpres = spark.sql("select period_as_date, qty_serv, product_line from team_group_quality.service_call_forecasting where product_line in ('Food Preservation')")
ncalls_overtime_foodpres = df_foodpres.groupby('period_as_date').agg({'qty_serv': 'sum'}).toPandas()
ncalls_overtime_foodpres['product_line'] = "Food Preservation"

In [0]:
ncalls_overtime_3 = pd.concat([ncalls_overtime_dishcare, ncalls_overtime_fabriccare, ncalls_overtime_foodprep, ncalls_overtime_foodpres], ignore_index=True)
result = sns.relplot(data=ncalls_overtime_3, x='period_as_date', y='sum(qty_serv)', kind='line', hue='product_line').set(title="# Calls over time split by product lines (major appliances)")

#### Let's see the number of calls over the years of PNC = 900272856 (picked from highest count(failure_date) above)

In [0]:
df_pncfocus1 = spark.sql("select period_as_date, qty_serv, pnc_sku from team_group_quality.service_call_forecasting where pnc_sku in ('900272856')")
ncalls_overtime_pncfocus1 = df_pncfocus1.groupby('period_as_date').agg({'qty_serv': 'sum'}).toPandas()
sns.relplot(data=ncalls_overtime_pncfocus1, x="period_as_date", y="sum(qty_serv)", kind='line').set(title="# Calls overtime for PNC = 900272856")

####Number of sold over time (overall)

In [0]:
df_sold_overtime = spark.sql("select extract(year from sell_out_date) sell_out_date_year, sell_out_date from team_group_quality.service_call_forecasting where sell_out_date between '2015' and '2023'")
nsold_overtime = df_sold_overtime.groupby('sell_out_date_year').agg({'sell_out_date': 'count'}).toPandas()
nsold_overtime = nsold_overtime.sort_values('sell_out_date_year', ascending=True)
nsold_overtime

In [0]:
sns.relplot(data=nsold_overtime, x="sell_out_date_year", y="count(sell_out_date)", kind='line').set(title="# sold over time")

###Number of sold over time with product line split

In [0]:
df_dishcare = spark.sql("select extract(year from sell_out_date) sell_out_date_year, sell_out_date, product_line from team_group_quality.service_call_forecasting where product_line in ('Dish Care') and sell_out_date between '2015' and '2023'")
nsold_overtime_dishcare = df_dishcare.groupby('sell_out_date_year').agg({'sell_out_date': 'count'}).toPandas()
nsold_overtime_dishcare['product_line'] = "Dish Care"

df_fabriccare = spark.sql("select extract(year from sell_out_date) sell_out_date_year, sell_out_date, product_line from team_group_quality.service_call_forecasting where product_line in ('Fabric Care') and sell_out_date between '2015' and '2023'")
nsold_overtime_fabriccare = df_fabriccare.groupby('sell_out_date_year').agg({'sell_out_date': 'count'}).toPandas()
nsold_overtime_fabriccare['product_line'] = "Fabric Care"

df_foodprep = spark.sql("select extract(year from sell_out_date) sell_out_date_year, sell_out_date, product_line from team_group_quality.service_call_forecasting where product_line in ('Food Preparation') and sell_out_date between '2015' and '2023'")
nsold_overtime_foodprep = df_foodprep.groupby('sell_out_date_year').agg({'sell_out_date': 'count'}).toPandas()
nsold_overtime_foodprep['product_line'] = "Food Preparation"

df_foodpres = spark.sql("select extract(year from sell_out_date) sell_out_date_year, sell_out_date, product_line from team_group_quality.service_call_forecasting where product_line in ('Food Preservation') and sell_out_date between '2015' and '2023'")
nsold_overtime_foodpres = df_foodpres.groupby('sell_out_date_year').agg({'sell_out_date': 'count'}).toPandas()
nsold_overtime_foodpres['product_line'] = "Food Preservation"

In [0]:
nsold_overtime_pl = pd.concat([nsold_overtime_dishcare, nsold_overtime_fabriccare, nsold_overtime_foodprep, nsold_overtime_foodpres], ignore_index=True)
result_nsold = sns.relplot(data=nsold_overtime_pl, x='sell_out_date_year', y='count(sell_out_date)', kind='line', hue='product_line').set(title="#sold overtime split by product lines (major appliances)")

#### Let's see the number of sold over the years of PNC = 900272856 (picked from highest count(failure_date) above)

In [0]:
df_pncfocus1 = spark.sql("select extract(year from sell_out_date) sell_out_date_year, sell_out_date, pnc_sku from team_group_quality.service_call_forecasting where pnc_sku in ('900272856')")
nsold_overtime_pncfocus1 = df_pncfocus1.groupby('sell_out_date_year').agg({'sell_out_date': 'count'}).toPandas()
sns.relplot(data=nsold_overtime_pncfocus1, x="sell_out_date_year", y="count(sell_out_date)", kind='line').set(title="# sold overtime for PNC = 900272856")

####TTFI Distribution

In [0]:
df_ttfi = spark.sql("select sell_out_date, period_as_date, qty_serv, ttfi from team_group_quality.service_call_forecasting")
ttfi = df_ttfi.groupby('ttfi').agg({'qty_serv': 'sum'}).toPandas()
ttfi = ttfi.sort_values('ttfi', ascending=True)
ttfi

In [0]:
sns.relplot(data=ttfi, x="ttfi", y="sum(qty_serv)", kind='line')